In [1]:
import pandas as pd 
import numpy as np
import re
from comaparison import check_po_date,check_doctype,check_po_number

In [2]:
IEPO_GROUND_data=pd.read_excel("IEPO_210_Documents.xlsx",sheet_name="IEPO_Cleaned")
IEPO_extracted_data=pd.read_csv("KA_Dump.csv")

In [3]:
IEPO_GROUND_data =IEPO_GROUND_data.drop_duplicates()

In [4]:
doc_ids=IEPO_GROUND_data["document_id"].unique()

IEPO_extracted_data = IEPO_extracted_data[
    IEPO_extracted_data["document_id"].astype(str).isin(doc_ids)
]

In [5]:
IEPO_extracted_data.loc[:, 'running_index'] =IEPO_extracted_data.groupby('document_id').cumcount() + 1
IEPO_extracted_data = IEPO_extracted_data.reset_index(drop=True)

#adjust the name of columns
IEPO_GROUND_data = IEPO_GROUND_data.rename(columns={"gt_po_no": "gt_po_number"})
IEPO_GROUND_data = IEPO_GROUND_data.rename(columns={"gt_doctype": "gt_doc_type"})

In [6]:
# Corrected code
IEPO_extracted_data = IEPO_extracted_data.drop_duplicates(subset="document_id")
IEPO_GROUND_data = IEPO_GROUND_data.drop_duplicates(subset="document_id")


In [7]:
special_character = r"[:|*|-|BE|()|-|(*)]"


def remove_special_characters(text):
    return re.sub(special_character, "", text)

In [8]:
IEPO_extracted_data = IEPO_extracted_data.astype(str)

IEPO_GROUND_data = IEPO_GROUND_data.astype(str)




for col in ["po_number_original", "po_number_corrected"]:
    IEPO_extracted_data[col] = IEPO_extracted_data[col].apply(remove_special_characters)
    
    
    
IEPO_GROUND_data["gt_po_number"] = IEPO_GROUND_data["gt_po_number"].apply(remove_special_characters)



for col in ["po_number_original", "po_number_corrected"]:
    IEPO_extracted_data[col] = IEPO_extracted_data[col].apply(remove_special_characters)
    
    
IEPO_GROUND_data["gt_po_number"] = IEPO_GROUND_data["gt_po_number"].apply(remove_special_characters)

In [9]:
Filtered_po_number=check_po_number(IEPO_GROUND_data,IEPO_extracted_data,"po_number" )

In [10]:
Filtered_doc_type=check_doctype(IEPO_GROUND_data,Filtered_po_number,"doc_type" )

In [11]:
Filtered_doc_type["DOCType_Check"].unique()

array(['Perfect Extraction', 'Business change+Extraction Error',
       'Perfect Extraction,Business Change'], dtype=object)

In [14]:
(Filtered_doc_type["DOCType_Check"] == 'Perfect Extraction').mean() * 100

77.60416666666666

In [13]:
Filtered_doc_type.to_csv("po_doc.csv",index=False)